# 01b_embedding_autoencoder

In this notebook, an autoencoder is trained on our training data textual description embeddings, reducing the embeddings to a bottlenecked 2-dimenstion latent space.

The purpose of this will be convert our textual project descriptions to two additional features we can use while training and testing our model.

In [1]:
import numpy as np
import seaborn as sns

from matplotlib import pyplot
import matplotlib.pylab as plt 
%matplotlib inline

from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.optimizers import Adam, SGD